1) Work out the Neyman-Pearson detection statistic for detecting a rectangular box located at position 555, with hight 1, size 100 in white gaussian noise (np.random.normal(0,1,10**6))
a) Write the statistical model for the two competing hypothesis (H0, H1)
b) If a false detection costs 10^6 dollars, but a true detection gains you a dollar, what would be the detection bar?
c) Suppose that the hight of the rectangular box is unknown, what would you do? Is the test optimal ?
d) Suppose that the noise was complex (as in complex numbers), and that the boxed signal had a uniform "hight" that is a complex number of unknown magnitude and phase. What would you do? is the test optimal?
e) Suppose that the position of the box is unknown. How would you detect it then?
How would you compute the detection statistic at all positions at once using FFT?
What would be the detection bar in this situation, for the same financial conditions as in (Q1 b)? Use monte-carlo to set the bar. What is the equivalent look-elsewhere-effect? 
f) What is the amplitude of the box such that if we "inject" a signal with this amplitude, the detection probability is 50% [what about 10%, 99%, 99.99%]
g) What would you do if the size of the box is unknown, and can take any width between 2 bins and 500 bins? Use monte-carlo to set the bar. What is the equivalent look-elsewhere-effect? 

2) You are looking for the same box as in (1), but suppose that the noise is of the following form:
n(t) = np.random.normal(0,1,10**6) convolved with a normalized triangular shape of width 500 [normalized such that np.linalg.norm(triangle)=1].
a) Write a statistical model for the null (H0) and the alternative hypothesis (H1) in real space what is the Neyman-Pearson detection statistic you would compute? [use matrix notation]
b) Write a statistical model for the null (H0) and the alternative hypothesis (H1)  in Fourier space. what is the Neyman-Pearson detection statistic you would compute? Is it the same statistic? Simulate a signal and compute the score in several positions, both in real space and Fourier space, make sure you get the same number up to machine precision.
c) Are you more sensitive with this noise source, or in Q1? What is the amplitude of the box such that if we "inject" a signal with this amplitude, the detection probability is 50% [what about 10%, 99%, 99.99%]?
(d) about the 50% detection amplitude as a function of the Triangle's width. Can you compute it analytically? (good approximation is OK) 

3) You are looking for the same box as in (1), but suppose that the noise is of the following form:
n(t) = np.random.normal(0,1,10**6) convolved with a normalized filter of width 500 and unknown shape [normalized such that np.linalg.norm(filter)=1].
a) Is it possible to obtain a good estimator of the filter? [Read about the Welch method AFTER trying to solve it yourself]
b) What is the impact of using the best-estimate filter in the detection statistic computed in (2) ? Is this hampering detection at all?
c) Suppose instead of 10**6 samples, you have only 10**4 samples, how does this impact the precision of estimating the filter?  Is this hampering detection at all?

4) Suppose you have the same situation as in (2), but after generating the noise, Gargamel chooses at random 10**3 samples and zeroizes them.
a) Compute the statistic from (2) in this situation, and plot their histogram. Is that the same histogram as in (2)? Would this interfere with detection?
b) Write the time-domain statistic relevant for detecting a signal at a particular place, taking the missing data into account.

In [ ]:
import numpy as np

1) Work out the Neyman-Pearson detection statistic for detecting a rectangular box located at position 555, with height 1, size 100 in white gaussian noise (np.random.normal(0,1,10**6))

********
a) $\textbf{Write the statistical model for the two competing hypothesis (H0, H1)}$
********

we have a series of $ n = 10^6 $ data points $x[i]$ dominated by white gaussian noise $w[i]$, in which we attempt to detect a signal $s[i]$ believed to be a rectangular box of height $h$ and size $l$ located at starting position $p$. 

thus $ w[i] \sim N(0,1) $

and  $ s[i] = h (\Theta[i - p] - \Theta[i - (p + l - 1)]) $
    
and we test the hypotheses

$H0:$ $ x[i] = w[i] $

$H1:$ $ x[i] = w[i] + s[i] $

with likelihoods 
$ L[H|x] = P[x|H] = P[H|x] \frac{P[x]}{P[H]} $

since we know that the probability of getting the draws $w[i]$ from $N(0,1)$ is

$ P[w] = (2 \pi)^{-\frac{n}{2}} \prod_{i}^{n} \exp(-\frac{w[i]^2}{2}) $ 

where the product is over all i, we can rewrite the hypotheses as 

$ H0:  w[i] = x[i] $

$ H1:  w[i] = x[i] - s[i] $
    
and plugging in these values of $w[i]$ gives us the likelihood ratio

$ \frac{L[H0]}{L[H1]} = \exp(-\frac{1}{2}  \sum_{i}^{n} x[i]^2 - (x[i] - s[i])^2) $

$ = \exp(\sum_{i}^{n} \frac{s[i]^2}{2} - s[i] x[i]) $

$ \equiv \Lambda(x) $

then we reject the null hypothesis $H0$ in favor of the alternative hypothesis $H1$ when

$ \Lambda(x) < \eta $

where the threshold (detection bar?) $\eta$ is set by the requirement that

$ P[\Lambda(x) < \eta | H0] = \alpha $

for our desired significance level (false detection rate) $\alpha$

the neyman-pearson lemma states that this test has the greatest power (smallest false negative / missed detection rate) of all statistical tests at level $\alpha$, but we notice that $ \sum_{i}^{n} s[i]^2 = h^2 l $ does not depend on the data, so it will help our computations to instead consider the monotonically transformed statistic

$ \sum_{i}^{n} s[i] x[i] \equiv \lambda(x) = -\ln{\Lambda} + \sum_{i}^{n} s[i]^2 $

now we have a new threshold $ \gamma \equiv -\ln[\Lambda] + h^2 l $ so that we reject $H0$ in favor of $H1$ when

$ \lambda(x) > \gamma $

where $\gamma$ is set to achieve our significance level

$ \alpha = P[\lambda(x) > \gamma | H0] $

next we find the distribution of our statistic,

$ \lambda(x) = \sum_{i}^{n} s[i] x[i] = h \sum_{i=p}^{p+l-1} x[i] $

which, under the null hypothesis $x[i] = w[i]$ is simply a sum of $l$ gaussian RVs $ h w[i] \sim N(0, h^2) $, resulting in a gaussian RV with zero mean and variance equal to $h^2 l$, i.e.,

$ \lambda(x) \sim N(0, h^2 l) \implies \frac{\lambda(x)}{h \sqrt{l}} \sim N(0, 1) $

thus, our false detection rate $ P[\lambda(x) > \gamma | H0] $ is the probability of a gaussian RV deviating from the mean by more than $ \frac{\gamma}{h \sqrt{l}} $ standard deviations

we can write this as

$ \alpha = \int_{\frac{\gamma}{h \sqrt{l}}}^{\infty} \exp(-\frac{y^2}{2}) dy $

which is equivalent to

$ 1 - \alpha = \int_{-\infty}^{\frac{\gamma}{h \sqrt{l}}} \exp(-\frac{y^2}{2}) dy = \frac{1}{2} ( 1 + Erf(\frac{\gamma}{h \sqrt{2 l}}) ) $

where $Erf[y]$ is the error function under Mathematica's conventions, so we can solve for our threshold in terms of our desired false detection rate using mathematica's InverseErf function:

$ \gamma = h \sqrt{2 l} * InverseErf(1 - 2 \alpha) $

********
b) $\textbf{If a false detection costs 10^6 dollars, but a true detection gains you a dollar, what would be the detection bar?}$
********

if the cost of false detection is $10^6$ times greater than the benefit of true detection, we set our false detection rate $ \alpha = 10^{-6} $ and get

$ \gamma = h \sqrt{2 l} * InverseErf(0.999998) = h \sqrt{l} * 4.75342 $

********
c) $\textbf{Suppose that the height of the rectangular box is unknown, what would you do? Is the test optimal ?}$
********

first note that the above test is equivalent to a threshold $ \gamma' \equiv \frac{\gamma}{h \sqrt{l}} = 4.75342 $ for a test which rejects $H0$ when $ \frac{\lambda(x)}{h \sqrt{l}} \equiv \lambda' > \gamma' $

thus, for any height $h$ we know that comparing the test statistic $ \lambda' = \frac{1}{\sqrt{l}} \sum_{i=p}^{p+l-1} x[i] $ to the detection bar $ \gamma' = 4.75342 $ will give us a false detection rate $ \alpha = 10^{-6} $

the Neyman-Pearson lemma tells us that this is the most powerful test for any $\alpha$, because $\lambda'$ is a sufficient statistic for the family of distributions parameterized by $h$

********
d) $\textbf{Suppose that the noise was complex (as in complex numbers),}$

$\textbf{and that the boxed signal had a uniform "height"} $

$\textbf{that is a complex number of unknown magnitude and phase.}$ 

$\textbf{What would you do? is the test optimal?}$
********

in this case, we have 2 components of data, and under both hypotheses the WGN and supposed "rectangular" signal of unknown complex "height" have components that are independent of each other, so we effectively have 2 independent opportunities to perform the same test (one on each component of the complex data, whether taken as $(Re[x],Im[x])$ or any other basis of the complex plane)

then we can use the same test described above, once for each component, and claim detection if either component's test statistic is above the threshold

however, since you would expect a false positive after $N$ independent tests each at false positive rate $\frac{1}{N}$, we should use false positive rate of approximately $\frac{\alpha}{2}$ for each test in order to preserve the overall rate of $\alpha$ after 2 tests

another way to see this is to recognize that this is the same problem as if we were to have $2n$ samples of a single data component, and we try to detect the signal (of known size $l$ but unknown height $h$) by testing $H0$ against $H1[position]$ at positions $p$ and $n+p$

more generally, if we have a null hypothesis consisting of a pair of independent parameters $(c_1,c_2)$ being equal to zero, and we consider it a detection if either one has a large enough test statistic, then tests with false positive rates $\alpha_1$ and $\alpha_2$ combine for an overall false positive rate of $ \alpha = 1 - (1-\alpha_1)(1-\alpha_2) = \alpha_1 + \alpha_2 - \alpha_1 \alpha_2 $

if we let $\alpha_1 = \alpha_2 = \alpha'$ then $\alpha = \alpha' (2 - \alpha')$ and we can solve for $\alpha' = 1 - \sqrt{1 - \alpha}$

this test is optimal in the sense that we are taking the largest possible $\alpha'$ given the desired overall $\alpha$, and we know that each of the two tests is the most powerful test of its respective component at level $\alpha'$, but the Neyman-Pearson lemma and Karlin-Rubin theorem don't guarantee that the overall test is uniformly most powerful for the two-parameter family of tests -- and in fact, it is ***NOT AN OPTIMAL TEST***

***A BETTER TEST*** can be constructed by recognizing that the supposed phase of the signal does not matter in the case of hypothesis testing against circularly symmetric WGN, so we should be able to construct a more powerful test that combines the real and imaginary components of the previous test into a threshold for the amplitude of a single complex test statistic

to make sure that we don't naively choose the wrong test statistic, let's return to the beginning of the problem and start with some new notation:

$ h = A e^{i \phi} $

***ALSO-- NOTATION CHANGE:***
let the number of samples (formerly $n$) now be called $N$ ($10^6$ in the example)

let $f[i]$ now be called $f_n$ with $ n = 0, 1, ..., N-1 $ and $ i = \sqrt{-1} $ always from now on

let the starting position (formerly $p$) now be called $n_s$ and the length (formerly $l$) now be called $\tau$

and let the ending position $ n_s + \tau - 1 \equiv n_{\tau} $

so, we have data $x_n$ and WGN $ w_n \sim CN(0,1) \implies \Re(w_n), \Im(w_n) \sim N(0, \frac{1}{2}) $

and we search for signal $ s_n = A e^{i \phi} ( \Theta_{n - n_s} - \Theta_{n - n_{\tau}} ) \equiv A e^{i \phi} \hat{s}_n $

using the hypotheses

$ H0: x_n = w_n $

$ H1: x_n = w_n + s_n $

this gives us likelihood functions

$ P[x_n | H0] = P[w_n = x_n] = \prod_n \frac{\exp(-|x_n|^2)}{\pi} $

$ P[x_n | H1] = P[w_n = x_n - s_n] = \prod_n \frac{\exp(-|x_n - s_n|^2)}{\pi} $

$ \implies \frac{P[x_n | H0]}{P[x_n | H1]} = \exp(\sum_n |x_n - s_n|^2 - |x_n|^2) = \exp(\sum_n |s_n|^2 - s_n^{*} x_n - x_n^{*} s_n) = \Lambda \implies $

$ \ln \Lambda = A^2 \tau - A \sum_n \hat{s}_n (e^{-i \phi} x_n + e^{i \phi} x_n^{*}) \implies $

$ A \tau - \frac{\ln \Lambda}{A} = \sum_n \hat{s}_n (e^{-i \phi} x_n + e^{i \phi} x_n^{*}) = 2 \sum_n \hat{s}_n \Re(e^{-i \phi} x_n) = 2 \sum_n \hat{s}_n |x_n| \cos(\arg(x_n) - \phi) $



so what do we do about $\phi$?

let's first note that

$ \sum_n \hat{s}_n \Re(e^{-i \phi} x_n) = \Re(e^{-i \phi} \sum_n \hat{s}_n x_n) = \Re(e^{-i \phi} z(x)) $

$ = |z| \cos(\arg(z) - \phi) = \frac{1}{2} (A \tau - \frac{\ln \Lambda}{A}) \equiv \lambda(x; \phi) $

where $ z(x) \equiv \sum_n \hat{s}_n x_n $

we see that $ \phi_{*} = 2 \pi j + \arg(z) $ maximizes $\lambda(x; \phi)$ for any integer $j$, giving

$ \lambda(x; \phi_{*}(x)) = |z| = |\sum_n \hat{s}_n x_n| $

since all values of $\phi$ are equally likely, and since our WGN is circularly symmetric, using the data to select the angle should not bias the statistic under the null hypothesis, and if the signal exists then it should offer our highest probability of detection (most powerful test)

under the null hypothesis we have $ \lambda(x | H0) = |\sum_n \hat{s}_n w_n| = |\sum_{n=n_s}^{n_\tau} w_n| $

so using $ w_n \sim CN(0,1) \implies \sum_{n=n_s}^{n_\tau} w_n \sim CN(0, \tau) $ tells us that $\lambda(x | H0)$ is the modulus of a circularly symmetric complex gaussian, which means that it should follow the Rayleigh distribution with variance $\frac{\tau}{2}$ and we have false alarm rate

$ \alpha = P[|z| > \eta | H0] = P[Rayleigh(\frac{\tau}{2}) > \eta] = 1 - CDF_{Rayleigh\frac{\tau}{2}}(\eta) = 1 - (1 - \exp(-\frac{\eta^2}{\tau})) = \exp(-\frac{\eta^2}{\tau}) $

so, for a given false alarm rate we use threshold

$ \eta(\alpha) = \sqrt{\tau \ln \frac{1}{\alpha}} $

and reject $H0$ when $ \lambda(x) = |\sum_n \hat{s}_n x_n| > \eta(\alpha) $

this should be the optimal test at level $\alpha$

********
e) $\textbf{Suppose that the position of the box is unknown. How would you detect it then?}$

$\textbf{How would you compute the detection statistic at all positions at once using FFT?}$

$\textbf{What would be the detection bar in this situation, for the same financial conditions as in (Q1 b)?}$

$\textbf{Use monte-carlo to set the bar. What is the equivalent look-elsewhere-effect? }$
********

we have test statistic $\lambda_{n_s}(x) = |\sum_n \hat{s}_{n, n_s, \tau} x_n| = |z_{n_s}(x)| $

we could do a test for each $ n_s = 1, 2, ..., N-1 $ where the index wraps around so that $ f_n \equiv f_{n mod N} $, but for each test we need to modify the individual false alarm rate by a some factor $LEF(N)$ that accounts for the look-elsewhere effect incurred from doing $N$ tests

naively we would have $ LEF(N) = \frac{1}{N} $, but even in the case of independent tests this is not exact, and in our case the tests are not independent

we note that $z$ has the form of a discrete convolution when written as

$ z_{n_s} = \sum_n \hat{s}_{n - n_s} x_n $

so $ F[z_{n_s}](k) = F[x_n](k) F[s_n](k) $ is the discrete fourier transform and and the inverse fourier transform gives

$ z_{n_s} = F^{-1}[F[\vec{x}] F[\vec{s}]]_{n_s} $

which can be computed efficiently using FFT

WAIT: convultion should be $ \sum_n \hat{s}_{n_s - n} x_n $ NEED TO ADJUST

********
f) $\textbf{What is the amplitude of the box such that if we "inject" a signal with this amplitude,}$

$\textbf{the detection probability is .5? (what about .1, .99, .9999?)}$
********

********
g) What would you do if the size of the box is unknown, and can take any width between 2 bins and 500 bins? Use monte-carlo to set the bar. What is the equivalent look-elsewhere-effect? 
********

********
2) You are looking for the same box as in (1), but suppose that the noise is of the following form:
n(t) = np.random.normal(0,1,10**6) convolved with a normalized triangular shape of width 500 [normalized such that np.linalg.norm(triangle)=1].
********

********
a) Write a statistical model for the null (H0) and the alternative hypothesis (H1) in real space what is the Neyman-Pearson detection statistic you would compute? [use matrix notation]
********

********
b) Write a statistical model for the null (H0) and the alternative hypothesis (H1)  in Fourier space. what is the Neyman-Pearson detection statistic you would compute? Is it the same statistic? Simulate a signal and compute the score in several positions, both in real space and Fourier space, make sure you get the same number up to machine precision.
********

********
c) Are you more sensitive with this noise source, or in Q1? What is the amplitude of the box such that if we "inject" a signal with this amplitude, the detection probability is 50% [what about 10%, 99%, 99.99%]?
********

********
(d) about the 50% detection amplitude as a function of the Triangle's width. Can you compute it analytically? (good approximation is OK) 
********

********
3) You are looking for the same box as in (1), but suppose that the noise is of the following form:
n(t) = np.random.normal(0,1,10**6) convolved with a normalized filter of width 500 and unknown shape [normalized such that np.linalg.norm(filter)=1].
********

********
a) Is it possible to obtain a good estimator of the filter? [Read about the Welch method AFTER trying to solve it yourself]
********

********
b) What is the impact of using the best-estimate filter in the detection statistic computed in (2) ? Is this hampering detection at all?
********

********
c) Suppose instead of 10**6 samples, you have only 10**4 samples, how does this impact the precision of estimating the filter?  Is this hampering detection at all?
********

********
4) Suppose you have the same situation as in (2), but after generating the noise, Gargamel chooses at random 10**3 samples and zeroizes them.
********

********
a) Compute the statistic from (2) in this situation, and plot their histogram. Is that the same histogram as in (2)? Would this interfere with detection?
********

********
b) Write the time-domain statistic relevant for detecting a signal at a particular place, taking the missing data into account.
********